In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from matplotlib import style
from sklearn import preprocessing, cross_validation, svm
from sklearn import linear_model 

In [2]:
devel=pd.read_csv('ComParE2017_devel.csv')
train=pd.read_csv('ComParE2017_train.csv')

In [3]:
train.head() 

,1,2,3,4,5,6,7,8,9,10,...,6365,6366,6367,6368,6369,6370,6371,6372,6373,y
0,2.32,0.0769,0.5600,0.554,0.666,1.010,0.112,0.347,0.458,0.3630,...,0.879,1.98,2.02,-18.5,0.458,91.3,49.6,78.1,49.1,0
1,2.86,0.9780,0.8390,0.776,1.620,2.340,0.846,0.723,1.570,0.5590,...,0.490,2.45,2.60,-13.5,0.437,155.0,79.7,132.0,52.4,1
2,2.03,0.7420,0.0000,0.932,1.160,1.450,0.230,0.285,0.515,0.2640,...,0.646,2.77,2.59,13.0,0.477,133.0,66.7,90.0,44.6,1
3,2.26,0.2500,0.9750,0.467,0.661,1.100,0.194,0.439,0.634,0.2780,...,0.605,2.33,2.42,-15.3,0.337,99.4,51.0,85.7,53.4,1
4,2.41,0.5730,0.0142,0.231,0.483,0.806,0.252,0.323,0.575,0.0575,...,0.450,2.00,1.97,18.2,0.512,99.5,44.6,85.2,36.9,0


In [4]:
X_tr = np.array(train.drop(['y'], 1))
X_dev = np.array(devel.drop(['y'], 1))
X_tr = preprocessing.scale(X_tr)
X_dev = preprocessing.scale(X_dev)
#df.fillna(value=-99999, inplace=True)
#df.dropna(inplace=True)
y_tr = np.array(train['y'])
y_dev = np.array(devel['y'])

C:\Users\Rahul\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [5]:
X_dev.shape

(3550, 6373)

In [6]:
# Feedforward neural net model
# split data into 80-20
from sklearn.cross_validation import KFold
from sklearn import cross_validation

# X_t, X_test, y_t, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

kf = KFold(3742, n_folds=5,shuffle=True)





In [7]:
#Perform batch SGD using manual backprop
kf_train_acc=[]
kf_val_acc=[]
D = X_tr.shape[1] #Number of features
K = max(y_tr)+1 #Number of classes assuming class index starts from 0

# Start with an initial set of parameters randomly
h = 10 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

# Initial values from hyperparameter
reg = 1e-3 # regularization strength
#Initial value for the Gradient Descent Parameter
step_size = 1e-2 #Also called learning rate

for train_index, val_index in kf:
#     print("TRAIN:", train_index, "VAL:", val_index)
    X_train, X_val = X_tr[train_index], X_tr[val_index]
    y_train, y_val = y_tr[train_index], y_tr[val_index]
    num_examples = X_train.shape[0]
    # gradient descent loop
    for i in range(100):

      # evaluate class scores, [N x K]
      hidden_layer = np.maximum(0, np.dot(X_train, W) + b) # note, ReLU activation
      scores = np.dot(hidden_layer, W2) + b2

      # compute the class probabilities
      exp_scores = np.exp(scores)
      probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

      # compute the loss: average cross-entropy loss and regularization
      corect_logprobs = -np.log(probs[range(num_examples),y_train])
      data_loss = np.sum(corect_logprobs)/num_examples
      reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
      loss = data_loss + reg_loss
      if i % 500 == 0:
        print ("iteration:",i, " loss:", loss)

      # compute the gradient on scores
      dscores = probs
      dscores[range(num_examples),y_train] -= 1
      dscores /= num_examples

      # backpropate the gradient to the parameters
      # first backprop into parameters W2 and b2
      dW2 = np.dot(hidden_layer.T, dscores)
      db2 = np.sum(dscores, axis=0, keepdims=True)
      # next backprop into hidden layer
      dhidden = np.dot(dscores, W2.T)
      # backprop the ReLU non-linearity
      dhidden[hidden_layer <= 0] = 0
      # finally into W,b
      dW = np.dot(X_train.T, dhidden)
      db = np.sum(dhidden, axis=0, keepdims=True)

      # add regularization gradient contribution
      dW2 += reg * W2
      dW += reg * W

      # perform a parameter update
      W += -step_size * dW
      b += -step_size * db
      W2 += -step_size * dW2
      b2 += -step_size * db2
    
      
    hidden_layer = np.maximum(0, np.dot(X_train, W) + b)
    scores = np.dot(hidden_layer, W2) + b2
    predicted_class_tr = np.argmax(scores, axis=1)
    print ("train accuracy: ",(np.mean(predicted_class_tr == y_train)))
    print("train recall: ",recall_score(y_train, predicted_class_tr))
    kf_train_acc.append(np.mean(predicted_class_tr == y_train))

    hidden_layer = np.maximum(0, np.dot(X_val, W) + b)
    scores = np.dot(hidden_layer, W2) + b2
    predicted_class_vl = np.argmax(scores, axis=1)
    print ("val accuracy: ",(np.mean(predicted_class_vl == y_val)))
    print("val recall: ",recall_score(y_val, predicted_class_vl))
    kf_val_acc.append(np.mean(predicted_class_vl == y_val))
    
print("cross validation-train acc:",np.mean(kf_train_acc),"cross validation-val acc:",np.mean(kf_val_acc))

iteration: 0  loss: 0.695421677123
train accuracy:  0.616438356164


NameError: name 'recall_score' is not defined

In [43]:
hidden_layer = np.maximum(0, np.dot(X_dev, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class_dv = np.argmax(scores, axis=1)
print ("dev accuracy: ",(np.mean(predicted_class_dv == y_dev)))

from sklearn.metrics import recall_score
dev_recall=recall_score(y_dev, predicted_class_dv, labels=None, pos_label=0, average='binary', sample_weight=None)
from sklearn.metrics import confusion_matrix
dev_conf_mtx=confusion_matrix(y_dev, predicted_class_dv, labels=None)
print(dev_recall,dev_conf_mtx,predicted_class_dv.shape)

dev accuracy:  0.630985915493
0.453216374269 [[ 620  748]
 [ 562 1620]] (3550,)


In [42]:
from sklearn.metrics import confusion_matrix
hidden_layer = np.maximum(0, np.dot(X_tr, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class_tn = np.argmax(scores, axis=1)
train_conf_mtx=confusion_matrix(y_tr, predicted_class_tn, labels=None)
train_conf_mtx

array([[1108,  332],
       [ 191, 2111]])